In [12]:
import cv2
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

# Configurações
INPUT_IMAGE_DIR = "processed_data/artDataset_clean"
OUTPUT_IMAGE_DIR = "processed_data2/artDataset_clean2"
CSV_PATH = "processed_data/artDataset_clean.csv"
os.makedirs(OUTPUT_IMAGE_DIR, exist_ok=True)

# Carregar o CSV para manter referência dos IDs
df = pd.read_csv(CSV_PATH)



def smart_artwork_crop(img_path, output_path, white_thresh=235, min_content_ratio=0.2):
    img = cv2.imread(img_path)
    if img is None:
        return False

    # Para molduras coloridas: converter para HSV e detectar tons claros
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask_hsv = cv2.inRange(hsv, (0, 0, 200), (180, 30, 255))  # Detecta brancos/bege

    # 1. Análise de bordas (combina HSV e grayscale para melhor detecção)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, mask_gray = cv2.threshold(gray, white_thresh, 255, cv2.THRESH_BINARY)

    # Combinar máscaras HSV e grayscale
    combined_mask = cv2.bitwise_or(mask_hsv, mask_gray)

    # 2. Remover pequenos ruídos
    kernel = np.ones((15,15), np.uint8)
    clean_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, kernel)

    # 3. Encontrar componentes conectados
    n_labels, labels, stats, _ = cv2.connectedComponentsWithStats(clean_mask)

    # 4. Identificar região central
    content_mask = np.zeros_like(gray)
    for i in range(1, n_labels):
        if stats[i, cv2.CC_STAT_AREA] > (img.size * 0.05):
            content_mask[labels == i] = 255

    # 5. Encontrar bounding box
    contours, _ = cv2.findContours(content_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Fallback: se não detectar nada válido, copia a imagem original
    if not contours:
        cv2.imwrite(output_path, img)
        return True

    main_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(main_contour)

    # 6. Verificação de área mínima
    if (w*h) < (img.shape[0]*img.shape[1]*min_content_ratio):
        cv2.imwrite(output_path, img)
        return True

    # 7. Cortar com margem de segurança (2% da imagem)
    margin = int(min(img.shape[:2]) * 0.02)
    x = max(0, x - margin)
    y = max(0, y - margin)
    w = min(img.shape[1] - x, w + 2*margin)
    h = min(img.shape[0] - y, h + 2*margin)

    cropped = img[y:y+h, x:x+w]
    cv2.imwrite(output_path, cropped)
    return True

# Processar todas as imagens com barra de progresso
success_count = 0
for idx, row in tqdm(df.iterrows(), total=len(df)):
    img_filename = f"image_{row['Unnamed: 0']+1}.png"
    input_path = os.path.join(INPUT_IMAGE_DIR, img_filename)
    output_path = os.path.join(OUTPUT_IMAGE_DIR, img_filename)

    if os.path.exists(input_path):
        if smart_artwork_crop(input_path, output_path):
            success_count += 1
    else:
        print(f"Arquivo não encontrado: {input_path}")

print(f"\nProcessamento concluído. {success_count}/{len(df)} imagens processadas com sucesso.")

# Atualizar CSV com caminhos das novas imagens (opcional)
df['image_clean_path'] = df['Unnamed: 0'].apply(lambda x: f"artDataset_clean/{x}.png")
df.to_csv("processed_data2/artDataset_clean2.csv", index=False)

  0%|          | 0/754 [00:00<?, ?it/s]

100%|██████████| 754/754 [00:12<00:00, 60.41it/s]


Processamento concluído. 754/754 imagens processadas com sucesso.
